In [1]:
import sys 
import random 
import matplotlib.pyplot as plt 
import numpy as np 
import math as m 
from datetime import datetime 
 
# précalcul du potentiel qui est linéaire 
RAPINIT = True 
# sous partie à mettre à jour à un nouveau pixel de taille 2S+1x2S+1 
S = 10 
MAJPETITE = False 
# 1% = 0.01, 0.1% = 0.001, convergence optimale à 0.00001 mais compromis vitesse 0.0001 
MAXDELTA = 0.3333333333333333
 
N=64
eta=1
    
#if len(sys.argv) != 3 : 
 #   N = 64 
  #  eta = 3 
#else: 
    # dimension de la grille NxN 
 #   N=int(sys.argv[1]) 
    # paramètre de croissance de la structure 
  #  eta=float(sys.argv[2]) 
    

partMax = 50000 
# la grille est de NxN mais en réalite la mesure d'une unité h est 1/N 
pixelsBitmap = [[0 for j in range(N)] for i in range(N)] 
# que pour le dessin pour éviter la transposition 
dessinBitmap = [[0 for j in range(N)] for i in range(N)] 
# conditions initiales -1 charge négative ou 1 charge positive ou 0 non defini sera mis à jour pendant le programme avec les nouvelles charges 
condInitBitmap = [[0 for j in range(N)] for i in range(N)] 
potentielsBitmap = [[0.0 for j in range(N)] for i in range(N)] 
pixelsCroissance = [] 
potentielsCroissance = [] 
 
def init(): 
    # ligne du bas en charge positives et trois charges supperposées en haut au milieu 
    # ligne du bas en charge positives et ligne du haut en charges négatives (nuage) 
    for i in range(N): 
        condInitBitmap[i][N-1] = 1 
        potentielsBitmap[i][N-1] = 1 
        dessinBitmap[N-1][i] = 1 
    for i in range(N): 
        condInitBitmap[i][0] = -1 
        potentielsBitmap[i][0] = 0 
        dessinBitmap[0][i] = 0.5 
    if RAPINIT: 
        # calcul analytique initial du potentiel entre deux plaques: potentiel linéaire en y 
        for j in range(1,N-1): 
            pot = 0.0+j*1.0/(N-1) 
            for i in range(N): 
                potentielsBitmap[i][j] = pot 
 
# paramètre de relaxation pour accélérer la convergence 
omega = 1.6 
 
def majPotentielRapide(i,j): 
    delta = 0.0 
    nb = 1 
    if abs(condInitBitmap[i][j]) != 1: 
        if i == 0: 
            delta = potentielsBitmap[i+1][j] + potentielsBitmap[i][j-1] + potentielsBitmap[i][j+1] - 3 * potentielsBitmap[i][j] 
            nb = 3 
        elif i == N-1: 
            nb = 3 
            delta = potentielsBitmap[i-1][j] + potentielsBitmap[i][j-1] + potentielsBitmap[i][j+1] - 3 * potentielsBitmap[i][j] 
        else: 
            delta = potentielsBitmap[i-1][j] + potentielsBitmap[i+1][j] + potentielsBitmap[i][j-1] + potentielsBitmap[i][j+1] - 4 * potentielsBitmap[i][j] 
            nb = 4 
             # on ne met à jour que les points qui ne sont pas des contraintes initiales ou qui ne continnent pas des charges 
            potentielsBitmap[i][j] = potentielsBitmap[i][j] + omega * delta / nb 
        return abs(delta/potentielsBitmap[i][j]/nb) 
    else:
        return 0.0 

def majPotentielsSousBitmapRapide(x,y,k): 
    # mise à jour de la sous partie centrée sur x,y de taille 2k+1 x 2k+1 
    maxdelta = 0 
    # parcours de bas en haut et de gauche à droite 
    # il ne faut pas dépasser le bord: on s'arrête avant 
    Ix = max(0,x-k) 
    Sx = min(N-1,x+k) 
    Iy = max(0,y-k) 
    Sy = min(N-1,y+k) 
    # N-1 à 0 
    for j in range(Sy,Iy-1,-1): 
        for i in range(Ix,Sx+1): 
            delta = majPotentielRapide(i,j) 
            if delta > maxdelta: 
                maxdelta = delta 
    return maxdelta 
 
def majPotentielsBitmapRapide(): 
    maxdelta = 0.0 
    # parcours de bas en haut et de gauche à droite 
    # N-1 à 0 
    for j in range(N-1,-1,-1): 
        # 0 à N-1 
        for i in range(N): 
            delta = majPotentielRapide(i,j) 
            if delta > maxdelta: 
                maxdelta = delta 
    return maxdelta 
 
def estValide(xy, bitmap): 
    # valide si qu'un seul voisin dessiné de xy dans bitmap 
    if bitmap[xy[0]+1][xy[1]] + bitmap[xy[0]-1][xy[1]] + bitmap[xy[0]][xy[1]+1] + bitmap[xy[0]][xy[1]-1] + bitmap[xy[0]+1][xy[1]+1] + bitmap[xy[0]-1][xy[1]+1] + bitmap[xy[0]+1][xy[1]-1] + bitmap[xy[0]-1][xy[1]-1] == 1: 
        return True 
    return False 
 
def sontAcotes(pt1,pt2): 
    if (abs(pt1[0] - pt2[0]) <= 1) and (abs(pt1[1] - pt2[1]) <= 1): 
        return True 
    return False 
 
def pasAcoteCroissance(pt, pixelsCroissance):
    for c in range(len(pixelsCroissance)): 
        if sontAcotes(pt, pixelsCroissance[c]): 
            return False 
    return True 
 
def eliminePixelsPres(pt, pixelsCroissance, potentielsCroissance): 
    for c in range(len(pixelsCroissance) - 1, -1, -1): 
        if sontAcotes(pt, pixelsCroissance[c]): 
            pixelsCroissance.pop(c) 
            potentielsCroissance.pop(c) 
    return None 
 
def estDansCroissance(pt, pixelsCroissance): 
    for c in range(len(pixelsCroissance)): 
        if pixelsCroissance[c] == pt: 
            return True 
    return False 
 
def croissanceRapide(nouv, pixelsBitmap, pixelsCroissance, potantielsCroissance): 
    # ajoute les pixels adjacents a nouv qui ne sont pas dans pixelsCroissance 
    candidats = [[nouv[0] + 1, nouv[1]], [nouv[0] - 1, nouv[1]], [nouv[0], nouv[1] + 1], [nouv[0], nouv[1] - 1], [nouv[0] + 1, nouv[1] + 1], [nouv[0] + 1, nouv[1] - 1], [nouv[0] - 1, nouv[1] + 1], [nouv[0] - 1, nouv[1] - 1], ] 
    for c in range(len(candidats)): 
        # si le candidat a qu'un seul voisin et n'est pas déjà un point dessiné et qu'il est pas en bordure supérieure 
        if estValide(candidats[c], pixelsBitmap) and (pixelsBitmap[candidats[c][0]][candidats[c][1]] == 0) and (candidats[c][1] > 0): 
 
            pixelsCroissance.append(candidats[c]) 
             
            potentielsCroissance.append(potentielsBitmap[candidats[c][0]][candidats[c][1]]) 
 
def choix(pixelsCroissance, potentielsCroissance): 
    choixPotentiel = random.uniform(0, 1) 
    PotentielTotalEta = 0 
    pTotal = [] 
    sommePTotal = 0 
    for i in range(len(potentielsCroissance)): 
        PotentielTotalEta += pow(potentielsCroissance[i], eta) 
    for i in range(len(pixelsCroissance)): 
        pi = pow(potentielsCroissance[i], eta) / PotentielTotalEta 
        sommePTotal += pi 
        pTotal.append(sommePTotal) 
    for i in range(len(pTotal)): 
        if pTotal[i] > choixPotentiel: 
            choixFinal = pixelsCroissance[i] 
            return([choixFinal, i]) 

def majPotentielsCroissance(pixelsCroissance, potentielsCroissance): 
    # mise a jour des potentiels de la liste complête 
    for c in range(len(pixelsCroissance)): 
        potentielsCroissance[c] = potentielsBitmap[pixelsCroissance[c][0]][pixelsCroissance[c][1]] 
 
def main(pixelsBitmap, pixelsCroissance, potentielsCroissance): 
    run = True 
    init() 
    # graine: début de décharge en haut au milieu 
    graine = [int(N / 2), 1] 
    nouv = graine 
    pixelsBitmap[nouv[0]][nouv[1]] = 1 
    dessinBitmap[nouv[1]][nouv[0]] = 1 
    condInitBitmap[nouv[0]][nouv[1]] = -1 
    potentielsBitmap[nouv[0]][nouv[1]] = 0 
    nbPart = 1 
    while run == True: 
        # elimine les pixelsCroissance qui sont proche de nouv 
        eliminePixelsPres(nouv, pixelsCroissance, potentielsCroissance) 
        # mise à jour de la carte des potentiels 
        maxdelta = 1 
        # condition d'arrêt à MAXDELTA du max des mises à jour des potentiels relatifs 
        cf = 0 
        if MAJPETITE: 
            while maxdelta > MAXDELTA: 
                maxdelta = majPotentielsSousBitmapRapide(nouv[0], nouv[1], S) 
                cf += 1
                
        maxdelta = 1 
        cs = 0 
        while maxdelta > MAXDELTA : 
            maxdelta = majPotentielsBitmapRapide() 
            print(cs, " : ", maxdelta)
            cs += 1 
        # définition des nouveaux sites de croissance potentiels 
        croissanceRapide(nouv, pixelsBitmap, pixelsCroissance, potentielsCroissance) 
        # mise à jour des potentiels des candidats à la croissance 
        majPotentielsCroissance(pixelsCroissance, potentielsCroissance) 
        res = choix(pixelsCroissance, potentielsCroissance) 
        nouv = res[0] 
        # le point choisi est elimine des croissances possibles 
        pixelsCroissance.pop(res[1]) 
        potentielsCroissance.pop(res[1]) 
            # le point choisi est dessine 
        pixelsBitmap[nouv[0]][nouv[1]] = 1 
        dessinBitmap[nouv[1]][nouv[0]] = 1 
        # le point choisi devient une contrainte à potentiel nul 
        condInitBitmap[nouv[0]][nouv[1]] = -1 
        potentielsBitmap[nouv[0]][nouv[1]] = 0 
        nbPart += 1 
        if nouv[0] == 0 or nouv[0] == N-1 or nouv[1] == N-2 or nbPart > partMax : 
            run = False 
 
    print("N=",N,"eta=",eta,"nbpart=",nbPart) 
 
main(pixelsBitmap, pixelsCroissance, potentielsCroissance) 
 
fichier = "foudre-"+str(N)+"-"+str(eta)+"-"+str(MAXDELTA)+"-"+datetime.now().strftime("%Y%m%d%H%M%S") 
np.save(fichier,np.array(dessinBitmap)) 


# Partie rajoutée


def Save_txt(y, N, fichier):
    y=np.array(dessinBitmap)
    space=''
    f=open(fichier + ".txt","a")
    for i in range(N):
        for j in range(N):
            if y[i][j]:
                print('#', end=space, file=f)
            else:
                print(' ', end=space, file=f)
        print("", file=f)
    f.close()

Save_txt(np.array(dessinBitmap), N, fichier)

0  :  0.5410447761194032
1  :  0.11904761904761903
0  :  0.5679465375087802
1  :  0.19076305220883533
0  :  4.32356686068776
1  :  0.32581094343951
0  :  18.348004659301825
1  :  0.6736304381009961
2  :  0.3018033941021706
0  :  1.3160140633564414
1  :  6.190657009577916
2  :  0.7698088833121729
3  :  0.15746441678118892
0  :  18.46630969129211
1  :  266.2756198933514
2  :  3.3869408465311746
3  :  2.0333416983535795
4  :  1.303997557097277
5  :  1.0406060714919902
6  :  0.19800178883016512
0  :  8.377297428666564
1  :  1.5150943148072082
2  :  0.6261354234786619
3  :  0.6582471706735201
4  :  65.3815983364379
5  :  0.6320891157163523
6  :  0.9660916046777012
7  :  0.24183028745536977
0  :  11.656960588968483
1  :  1.9932365907145406
2  :  1.7208388393999436
3  :  10.242835141637485
4  :  2.8759695918576784
5  :  8.657632690935747
6  :  0.9769627489238142
7  :  0.5248851062541351
8  :  0.6274076051080584
9  :  0.17510933637773127
0  :  4.02359400349651
1  :  1.2713556938799782
2  :  0.

8  :  0.7502549006376487
9  :  1.2609500296547842
10  :  0.385557947008627
11  :  0.21591029780251883
0  :  8.127780778111859
1  :  3.1515419173871115
2  :  9.553920933996784
3  :  53.67860980255057
4  :  1.0673797146248678
5  :  4.4986443978427655
6  :  1.4889840733305855
7  :  1.4534927152339099
8  :  5.587861303849218
9  :  1.0436786912452582
10  :  0.12631099232643972
0  :  5.1818978465191385
1  :  6.352735486055861
2  :  8.62898221509066
3  :  55.58476605240064
4  :  4.938038499301244
5  :  1.877909794912194
6  :  2.470245409321195
7  :  1.2798537630103095
8  :  1.1975381746851972
9  :  6.491498404675554
10  :  0.9669282818436615
11  :  0.19591007993439058
0  :  2.844536921277082
1  :  1.2192620360223028
2  :  1.0498736822555392
3  :  1.0108871241060553
4  :  0.30276622229330963
0  :  4.803640083314095
1  :  6.109583526751191
2  :  8.51985301457586
3  :  16.325306506746177
4  :  7.86432278742494
5  :  6.820967477594774
6  :  135.66393391895124
7  :  135.66393391895124
8  :  14.143

9  :  2.5959656783331346
10  :  4.260904805960804
11  :  1.9889474031698202
12  :  0.684929729232585
13  :  0.18518654168354828
0  :  0.6819525173537874
1  :  0.29225751239236264
0  :  2.8549224877040014
1  :  1.2149481515636662
2  :  4.36491944339734
3  :  8.667177717154063
4  :  0.5748141178526179
5  :  0.9263418454230903
6  :  1.0423383111555498
7  :  0.35369236646310953
8  :  0.48601702443084754
9  :  7.751001982977266
10  :  0.5647293558240571
11  :  0.3947455704881238
12  :  0.09361014139231519
0  :  1.1069846403920973
1  :  7.1891962268349925
2  :  9.398298130601882
3  :  2.7776481034431972
4  :  4.4359459928365
5  :  22.072180519523418
6  :  82.30388185656088
7  :  20.66859837743778
8  :  21.50432957260394
9  :  22.311019068938982
10  :  17.23096291731881
11  :  19.346350706679022
12  :  13.29161877792864
13  :  45.50566272359136
14  :  17.76021907731715
15  :  10.904882084513495
16  :  34.28933365719157
17  :  12.940308620749308
18  :  2.02279609003416
19  :  1.537388048789852

6  :  19.56900801576211
7  :  67.70934145618455
8  :  6.89914253439998
9  :  5.897164301636443
10  :  7.797064091662179
11  :  19.037340710296636
12  :  4.396901549005922
13  :  3.302765438487801
14  :  1.424088922144094
15  :  1.2581805177649772
16  :  7.752336808421931
17  :  0.8504518468587469
18  :  0.15428799705430815
0  :  4.048630818309603
1  :  1.6498668133303018
2  :  4.28461130407104
3  :  0.8837905036839011
4  :  0.8485195409240988
5  :  0.5262805574429185
6  :  0.21201445278302097
0  :  18.72914481464902
1  :  308.8479024872106
2  :  17.441077179484438
3  :  95.37918455741803
4  :  95.6357145254686
5  :  15.515874994850387
6  :  103.14979014893693
7  :  12.685454354176324
8  :  12.252208242775154
9  :  24.51414093878017
10  :  9.257800030350417
11  :  8.439117447977582
12  :  3.5776929489742826
13  :  22.12376460386002
14  :  15.150536750274583
15  :  32.11597072175249
16  :  4.582900807541787
17  :  2.7909589061329756
18  :  3.3978310618244336
19  :  1.1550613586082163
20 

5  :  1.0714171579373457
6  :  0.4538655226650711
7  :  8.560954132168249
8  :  1.1592929551125923
9  :  2.547979200844295
10  :  2.176891039753749
11  :  1.9173681968137704
12  :  5.996630760014941
13  :  15.524833269010667
14  :  1.6941162326350125
15  :  3.6700622057526235
16  :  2.5018263640132496
17  :  1.2275540402849416
18  :  4.4707879899434095
19  :  69.50927354651157
20  :  3.842184612075497
21  :  1.9852638252238302
22  :  1.1053904600714977
23  :  2.2596511220872846
24  :  0.21355125371584371
0  :  1.4070904467061838
1  :  4.701655082155901
2  :  3.880111009857346
3  :  3.8155592990717953
4  :  4.380303198348825
5  :  13.095521896579559
6  :  82.87145502483186
7  :  22.041673791992547
8  :  4.684973132608225
9  :  6.778418020907248
10  :  3.3070961726415438
11  :  3.21181986689195
12  :  1.565034024079851
13  :  25.2802257640094
14  :  0.7903247539637881
15  :  0.767706427699699
16  :  13.063735117646853
17  :  0.6854245725067082
18  :  0.3936851466287421
19  :  0.076852085

...

2736  :  0.3333333333333334
2737  :  0.3333333333333334
2738  :  0.3333333333333334
2739  :  0.3333333333333334
2740  :  0.3333333333333334
2741  :  0.3333333333333334
2742  :  0.3333333333333334
2743  :  0.3333333333333334
2744  :  0.3333333333333334
2745  :  0.3333333333333334
2746  :  0.3333333333333334
2747  :  0.3333333333333334
2748  :  0.3333333333333334
2749  :  0.3333333333333334
2750  :  0.3333333333333334
2751  :  0.3333333333333334
2752  :  0.3333333333333334
2753  :  0.3333333333333334
2754  :  0.3333333333333334
2755  :  0.3333333333333334
2756  :  0.3333333333333334
2757  :  0.3333333333333334
2758  :  0.3333333333333334
2759  :  0.3333333333333334
2760  :  0.3333333333333334
2761  :  0.3333333333333334
2762  :  0.3333333333333334
2763  :  0.3333333333333334
2764  :  0.3333333333333334
2765  :  0.3333333333333334
2766  :  0.3333333333333334
2767  :  0.3333333333333334
2768  :  0.3333333333333334
2769  :  0.3333333333333334
2770  :  0.3333333333333334
2771  :  0.333333333

3086  :  0.3333333333333334
3087  :  0.3333333333333334
3088  :  0.3333333333333334
3089  :  0.3333333333333334
3090  :  0.3333333333333334
3091  :  0.3333333333333334
3092  :  0.3333333333333334
3093  :  0.3333333333333334
3094  :  0.3333333333333334
3095  :  0.3333333333333334
3096  :  0.3333333333333334
3097  :  0.3333333333333334
3098  :  0.3333333333333334
3099  :  0.3333333333333334
3100  :  0.3333333333333334
3101  :  0.3333333333333334
3102  :  0.3333333333333334
3103  :  0.3333333333333334
3104  :  0.3333333333333334
3105  :  0.3333333333333334
3106  :  0.3333333333333334
3107  :  0.3333333333333334
3108  :  0.3333333333333334
3109  :  0.3333333333333334
3110  :  0.3333333333333334
3111  :  0.3333333333333334
3112  :  0.3333333333333334
3113  :  0.3333333333333334
3114  :  0.3333333333333334
3115  :  0.3333333333333334
3116  :  0.3333333333333334
3117  :  0.3333333333333334
3118  :  0.3333333333333334
3119  :  0.3333333333333334
3120  :  0.3333333333333334
3121  :  0.333333333

3434  :  0.3333333333333334
3435  :  0.3333333333333334
3436  :  0.3333333333333334
3437  :  0.3333333333333334
3438  :  0.3333333333333334
3439  :  0.3333333333333334
3440  :  0.3333333333333334
3441  :  0.3333333333333334
3442  :  0.3333333333333334
3443  :  0.3333333333333334
3444  :  0.3333333333333334
3445  :  0.3333333333333334
3446  :  0.3333333333333334
3447  :  0.3333333333333334
3448  :  0.3333333333333334
3449  :  0.3333333333333334
3450  :  0.3333333333333334
3451  :  0.3333333333333334
3452  :  0.3333333333333334
3453  :  0.3333333333333334
3454  :  0.3333333333333334
3455  :  0.3333333333333334
3456  :  0.3333333333333334
3457  :  0.3333333333333334
3458  :  0.3333333333333334
3459  :  0.3333333333333334
3460  :  0.3333333333333334
3461  :  0.3333333333333334
3462  :  0.3333333333333334
3463  :  0.3333333333333334
3464  :  0.3333333333333334
3465  :  0.3333333333333334
3466  :  0.3333333333333334
3467  :  0.3333333333333334
3468  :  0.3333333333333334
3469  :  0.333333333

3771  :  0.3333333333333334
3772  :  0.3333333333333334
3773  :  0.3333333333333334
3774  :  0.3333333333333334
3775  :  0.3333333333333334
3776  :  0.3333333333333334
3777  :  0.3333333333333334
3778  :  0.3333333333333334
3779  :  0.3333333333333334
3780  :  0.3333333333333334
3781  :  0.3333333333333334
3782  :  0.3333333333333334
3783  :  0.3333333333333334
3784  :  0.3333333333333334
3785  :  0.3333333333333334
3786  :  0.3333333333333334
3787  :  0.3333333333333334
3788  :  0.3333333333333334
3789  :  0.3333333333333334
3790  :  0.3333333333333334
3791  :  0.3333333333333334
3792  :  0.3333333333333334
3793  :  0.3333333333333334
3794  :  0.3333333333333334
3795  :  0.3333333333333334
3796  :  0.3333333333333334
3797  :  0.3333333333333334
3798  :  0.3333333333333334
3799  :  0.3333333333333334
3800  :  0.3333333333333334
3801  :  0.3333333333333334
3802  :  0.3333333333333334
3803  :  0.3333333333333334
3804  :  0.3333333333333334
3805  :  0.3333333333333334
3806  :  0.333333333

KeyboardInterrupt: 